# San Francisco Housing Rental Analysis

In this assignment, you will perform basic analysis for the San Francisco Housing Market to allow potential real estate investors to choose rental investment properties. 

In [89]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from dotenv import load_dotenv

%matplotlib inline

In [90]:
# Read the Mapbox API key
load_dotenv()
mapbox_token = os.getenv("pk.eyJ1IjoibWlhbGkxOTg3IiwiYSI6ImNrZ21wZ2M2aTJybTgycnBkbnNyNWt0czcifQ.Q2oBq9u7NUnTAgCL7Zs1AA")
px.set_mapbox_access_token(mapbox_token)

## Load Data

In [91]:
# Read the census data into a Pandas DataFrame
sfo_data = pd.read_csv('sfo_neighborhoods_census_data.csv', index_col="year")
sfo_data.head()

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
year,,,,
2010,Alamo Square,291.182945,372560,1239
2010,Anza Vista,267.932583,372560,1239
2010,Bayview,170.098665,372560,1239
2010,Buena Vista Park,347.394919,372560,1239
2010,Central Richmond,319.027623,372560,1239


- - - 

## Housing Units Per Year

In this section, you will calculate the number of housing units per year and visualize the results as a bar chart using the Pandas plot function. 

Hint: Use the Pandas groupby function

Optional challenge: Use the min, max, and std to scale the y limits of the chart.

In [92]:
sfo_housing_units_data = sfo_data.groupby(by='year')['housing_units'].mean()
sfo_housing_units_data

year
2010    372560
2011    374507
2012    376454
2013    378401
2014    380348
2015    382295
2016    384242
Name: housing_units, dtype: int64

In [93]:
max_sfo_housing_units_data =sfo_housing_units_data.max()
min_sfo_housing_units_data = sfo_housing_units_data.min()-5000

In [94]:
sfo_housing_units_data.hvplot.bar(ylim=(min_sfo_housing_units_data,max_sfo_housing_units_data), title ="Housing Units in SF from 2010 to 2016",yformatter="%.0f")

:Bars   [year]   (housing_units)

- - - 

## Average Prices per Square Foot

In this section, you will calculate the average gross rent and average sales price for each year. Plot the results as a line chart.

### Average Gross Rent in San Francisco Per Year

In [95]:
sfo_avg_data = sfo_data.groupby(by='year').mean().drop(columns=['housing_units'])
sfo_avg_data

,sale_price_sqr_foot,gross_rent
year,,
2010,369.344353,1239
2011,341.903429,1530
2012,399.389968,2324
2013,483.600304,2971
2014,556.277273,3528
2015,632.540352,3739
2016,697.643709,4390


In [96]:
sfo_avg_data.hvplot.line(x='year',y='gross_rent',ylabel='Gross Rent',title='Avg Gross Rent in San Francisco')

:Curve   [year]   (gross_rent)

### Average Sales Price per Year

In [97]:
sfo_avg_data.hvplot.line(x='year',y='sale_price_sqr_foot',ylabel='Avg Sale Price',title='Avg Sales Price in San Francisco')

:Curve   [year]   (sale_price_sqr_foot)

- - - 

## Average Prices by Neighborhood

In this section, you will use hvplot to create an interactive visulization of the Average Prices with a dropdown selector for the neighborhood.

Hint: It will be easier to create a new DataFrame from grouping the data and calculating the mean prices for each year and neighborhood

In [98]:
neighborhood_avg_data = sfo_data.groupby(['year','neighborhood']).mean()
neighborhood_avg_dataframe = pd.DataFrame(neighborhood_avg_data).reset_index()
neighborhood_avg_dataframe

,year,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,2010,Alamo Square,291.182945,372560,1239
1,2010,Anza Vista,267.932583,372560,1239
2,2010,Bayview,170.098665,372560,1239
3,2010,Buena Vista Park,347.394919,372560,1239
4,2010,Central Richmond,319.027623,372560,1239
...,...,...,...,...,...
392,2016,Telegraph Hill,903.049771,384242,4390
393,2016,Twin Peaks,970.085470,384242,4390
394,2016,Van Ness/ Civic Center,552.602567,384242,4390
395,2016,Visitacion Valley,328.319007,384242,4390


In [108]:
neighborhood_avg_dataframe.hvplot(groupby='neighborhood',y='sale_price_sqr_foot',x='year')

:DynamicMap   [neighborhood]
   :Curve   [year]   (sale_price_sqr_foot)

- - - 

## The Top 10 Most Expensive Neighborhoods

In this section, you will need to calculate the mean sale price for each neighborhood and then sort the values to obtain the top 10 most expensive neighborhoods on average. Plot the results as a bar chart.

In [100]:
neighborhood_avg_data = sfo_data.groupby(by='neighborhood').mean().nlargest(10, 'sale_price_sqr_foot').reset_index()
neighborhood_avg_data

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,Union Square District,903.993258,377427.50,2555.166667
1,Merced Heights,788.844818,380348.00,3414.000000
2,Miraloma Park,779.810842,375967.25,2155.250000
3,Pacific Heights,689.555817,378401.00,2817.285714
4,Westwood Park,687.087575,382295.00,3959.000000
5,Telegraph Hill,676.506578,378401.00,2817.285714
6,Presidio Heights,675.350212,378401.00,2817.285714
7,Cow Hollow,665.964042,378401.00,2817.285714
8,Potrero Hill,662.013613,378401.00,2817.285714
9,South Beach,650.124479,375805.00,2099.000000


In [101]:
neighborhood_avg_data.hvplot.bar(x='neighborhood',y='sale_price_sqr_foot')

:Bars   [neighborhood]   (sale_price_sqr_foot)

- - - 

## Parallel Coordinates and Parallel Categories Analysis

In this section, you will use plotly express to create parallel coordinates and parallel categories visualizations so that investors can interactively filter and explore various factors related to the sales price of the neighborhoods. 

Using the DataFrame of Average values per neighborhood (calculated above), create the following visualizations:
1. Create a Parallel Coordinates Plot
2. Create a Parallel Categories Plot

In [102]:
px.parallel_coordinates(neighborhood_avg_dataframe, color='sale_price_sqr_foot')

In [103]:
px.parallel_categories(
    neighborhood_avg_dataframe,
    dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units","gross_rent"],
    color="sale_price_sqr_foot",
    color_continuous_scale=px.colors.sequential.Inferno,
    labels={
        "neightborhood": "Neighborhood",
        "sale_price_sqr_foot": "Sale Price / Foot",
        "housing_units": "Housing Units",
        "gross_rent": "Gross Rent"
    },
)

- - - 

## Neighborhood Map

In this section, you will read in neighboor location data and build an interactive map with the average prices per neighborhood. Use a scatter_mapbox from plotly express to create the visualization. Remember, you will need your mapbox api key for this.

### Load Location Data

In [104]:
# Load neighborhoods coordinates data
df_neighborhood_locations = pd.read_csv('neighborhoods_coordinates.csv')
df_neighborhood_locations.head()

,Neighborhood,Lat,Lon
0,Alamo Square,37.791012,-122.402100
1,Anza Vista,37.779598,-122.443451
2,Bayview,37.734670,-122.401060
3,Bayview Heights,37.728740,-122.410980
4,Bernal Heights,37.728630,-122.443050


### Data Preparation

You will need to join the location data with the mean prices per neighborhood

1. Calculate the mean values for each neighborhood
2. Join the average values with the neighborhood locations

In [105]:
neighborhood_avg_data2 = sfo_data.groupby(by='neighborhood').mean().reset_index()
neighborhood_avg_data2

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent
0,Alamo Square,366.020712,378401.00,2817.285714
1,Anza Vista,373.382198,379050.00,3031.833333
2,Bayview,204.588623,376454.00,2318.400000
3,Bayview Heights,590.792839,382295.00,3739.000000
4,Bernal Heights,576.746488,379374.50,3080.333333
...,...,...,...,...
68,West Portal,498.488485,376940.75,2515.500000
69,Western Addition,307.562201,377427.50,2555.166667
70,Westwood Highlands,533.703935,376454.00,2250.500000
71,Westwood Park,687.087575,382295.00,3959.000000


In [106]:
mapdata = pd.concat([ df_neighborhood_locations, neighborhood_avg_data2], axis="columns", join="inner").drop(columns=['neighborhood'])
mapdata

,Neighborhood,Lat,Lon,sale_price_sqr_foot,housing_units,gross_rent
0,Alamo Square,37.791012,-122.402100,366.020712,378401.00,2817.285714
1,Anza Vista,37.779598,-122.443451,373.382198,379050.00,3031.833333
2,Bayview,37.734670,-122.401060,204.588623,376454.00,2318.400000
3,Bayview Heights,37.728740,-122.410980,590.792839,382295.00,3739.000000
4,Bernal Heights,37.728630,-122.443050,576.746488,379374.50,3080.333333
...,...,...,...,...,...,...
68,West Portal,37.740260,-122.463880,498.488485,376940.75,2515.500000
69,Western Addition,37.792980,-122.435790,307.562201,377427.50,2555.166667
70,Westwood Highlands,37.734700,-122.456854,533.703935,376454.00,2250.500000
71,Westwood Park,37.734150,-122.457000,687.087575,382295.00,3959.000000


### Mapbox Visualization

Plot the aveage values per neighborhood with a plotly express scatter_mapbox visualization.

In [107]:
map = px.scatter_mapbox(
    mapdata,
    lat="Lat",
    lon="Lon",
    size="sale_price_sqr_foot",
    color="Neighborhood",
    zoom=4
)

map.show()